In [4]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os 
import tensorflow as tf
from imutils import paths
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [5]:
dataset = r"C:\Users\hassa\Documents\python\face mask detection\data"

In [6]:
imagePaths=list(paths.list_images(dataset))

In [7]:
batch_size = 50
img_height = 200
img_width = 200

In [8]:
## loading training data
training_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset,
    validation_split=0.2,
    subset= "training",
    seed=42,
    image_size= (img_height, img_width),
    batch_size=batch_size
)

Found 7553 files belonging to 2 classes.
Using 6043 files for training.


In [9]:
## loading testing data
testing_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset,
    validation_split=0.2,
    subset= "validation",
    seed=42,
    image_size= (img_height, img_width),
    batch_size=batch_size
)

Found 7553 files belonging to 2 classes.
Using 1510 files for validation.


In [10]:
class_names = training_ds.class_names
class_names

['with_mask', 'without_mask']

In [11]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
training_ds = training_ds.cache().prefetch(buffer_size=AUTOTUNE)
testing_ds = testing_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [12]:
AUTOTUNE
training_ds
testing_ds

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 200, 200, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [13]:
## lets define our CNN
model = tf.keras.models.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(128, 3, activation='relu'),
  layers.MaxPooling2D(),
  # layers.Flatten(),
  layers.GlobalAveragePooling2D(),
  layers.Dense(256, activation='relu'),
  layers.Dense(2, activation= 'softmax')
])

In [14]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
## lets train our CNN
retVal = model.fit(training_ds, validation_data= testing_ds, epochs = 5)

Epoch 1/5
 71/121 [================>.............] - ETA: 2:39 - loss: 0.5945 - accuracy: 0.6752

In [18]:
plt.plot(retVal.history['loss'], label = 'training loss')
plt.plot(retVal.history['accuracy'], label = 'training accuracy')
plt.legend()

NameError: name 'retVal' is not defined

In [19]:
model.save('maskDetector.model')

ValueError: Model <keras.engine.sequential.Sequential object at 0x0000015274DAE0B0> cannot be saved because the input shape is not available. Please specify an input shape either by calling `build(input_shape)` directly, or by calling the model on actual data using `Model()`, `Model.fit()`, or `Model.predict()`.